In [1]:
import cloudpickle
from scipy.sparse import dok_matrix
from scipy.sparse import tril
import scipy

def load_dict(path,zeilen,spalten):
    if(zeilen >= spalten):
        template = "blockcounts_{i}_{j}".format(i=zeilen,j=spalten)
    else:
        template = "blockcounts_{i}_{j}".format(i=spalten,j=zeilen)

    file_path = path + '\\' + template
    co_occurences = {}
    
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            match = re.match('\(([0-9]{1,}), ([0-9]{1,})\):([0-9.]{1,})',line)
            x = int(match.group(1))
            y = int(match.group(2))
            count = float(match.group(3))
            if (x,y) in co_occurences:
                co_occurences[(x,y)] += count
            else:
                co_occurences[(x,y)] = count
    
    return co_occurences


def load_co_occurence(path,zeilen,spalten):
    co_occurences_dict = load_dict(path,zeilen,spalten)
    print(len(co_occurences_dict))
    coocurrence = dok_matrix((20000,20000),dtype='d')
    
    coocurrence._update(co_occurences_dict)
    
    coocurrence  = coocurrence.toarray()
    zero = tf.constant(0, dtype=tf.float32)
    where = tf.not_equal(coocurrence, zero)
    indices = tf.where(where)
    print(indices.shape[0])
    
    if spalten > zeilen :
        raise Exception("You sha'll not get symmetrical context")
        #may need to fix this in the future due to other non symmetrical feature
    
    if spalten == zeilen:
        print('mirroring')
        #print(coocurrence.toarray())
        coocurrence = coocurrence + tril(coocurrence,k=-1).transpose()
    return coocurrence

In [2]:
import re
import os
import h5py
import numpy as np
import sys
sys.path.append('..')
from Vocabulary import *
import tensorflow as tf

# The default block length of 20000 must be divisible by this number. 
split_length = 4 #2,4,5,10
size = int(20000/split_length)

path = 'E:\\tmp\\coocurrence_blocks_m'
path_hdf = r'E:\\tmp\\hdf_m'
regex = r'blockcounts_([0-9]{1,})_([0-9]{1,})'

vocab = Vocabulary()
vocab.load('..\\vocabs\\m_base')
unvocab = Vocabulary()
unvocab.load('..\\vocabs\\unfilteredm_base')
onlyDiag = True

for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i,j   = int(match.group(1)), int(match.group(2))
    print('output for progress ' + str(i)+','+str(j))
    if(i < j):
        continue
        
    if (i != j and onlyDiag):
        continue
    
    co_occurence = load_co_occurence(path,i,j)
    
    
    if(i == j):
        print('start diag')
        start_index = 20000 * int(i)
        for diagonal in range(20000):
            try:
                word = vocab.id2Word[start_index + diagonal]
                count = unvocab.word_frequency[word]
                co_occurence[diagonal,diagonal] = count
            except: 
                pass
        print('end diag')
    
    for sub_i in range(split_length):
        for sub_j in range(split_length):
            a,b = i*split_length+sub_i , j*split_length+sub_j
            print(a,b)
            filename = 'tf_cooccurence_{a}_{b}_withDiag.hdf'.format(a = a,b = b)
            
            f = h5py.File( path_hdf + '\\' +filename, "w")#plus experiment name
            co_occurence_hdf = f.create_dataset("co-ocurrence", (size, size))
            part = co_occurence[sub_i*size:(sub_i+1)*size,sub_j*size:(sub_j+1)*size]
            co_occurence_hdf[:,:] = part
            f.close()
        

output for progress 0,0
1415538
1415538
mirroring
start diag
end diag
0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3
output for progress 10,0
output for progress 10,1
output for progress 10,10
1454917
1454917
mirroring
start diag
end diag
40 40
40 41
40 42
40 43
41 40
41 41
41 42
41 43
42 40
42 41
42 42
42 43
43 40
43 41
43 42
43 43
output for progress 10,2
output for progress 10,3
output for progress 10,4
output for progress 10,5
output for progress 10,6
output for progress 10,7
output for progress 10,8
output for progress 10,9
output for progress 11,0
output for progress 11,1
output for progress 11,10
output for progress 11,11
1621070
1621070
mirroring
start diag
end diag
44 44
44 45
44 46
44 47
45 44
45 45
45 46
45 47
46 44
46 45
46 46
46 47
47 44
47 45
47 46
47 47
output for progress 11,2
output for progress 11,3
output for progress 11,4
output for progress 11,5
output for progress 11,6
output for progress 11,7
output for progress 11,8
output for progress 11,9
outp